<a href="https://colab.research.google.com/github/Filoferro03/ActionSpottingCalcio/blob/main/TesiVisioneArtificialeActionSpotting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Celle da eseguire ad ogni riavvio



In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [1]:
# Clona il repository del progetto
!git clone https://github.com/OpenSportsLab/OSL-ActionSpotting.git
%cd OSL-ActionSpotting

# Installa tutte le dipendenze, inclusa SoccerNet
!pip install -r requirements.txt
!pip install -e .
!pip install --extra-index-url https://pypi.nvidia.com/ nvidia-dali-cuda120

fatal: destination path 'OSL-ActionSpotting' already exists and is not an empty directory.
/content/OSL-ActionSpotting
Obtaining file:///content/OSL-ActionSpotting
  Preparing metadata (setup.py) ... done
  Attempting uninstall: oslactionspotting
    Found existing installation: oslactionspotting 0.2.4
    Uninstalling oslactionspotting-0.2.4:
      Successfully uninstalled oslactionspotting-0.2.4
  Running setup.py develop for oslactionspotting
Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.9/444.9 MB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.3/23.3 MB 125.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 149.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 131.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 182.6 MB/s eta 0:00:00


In [2]:
# Cella 3: Download, Decompressione e Correzione Dati (Eseguire solo la prima volta)

import os
import zipfile
from SoccerNet.Downloader import SoccerNetDownloader

# Definisci i percorsi
soccernet_root = "/content/gdrive/MyDrive/datasets/SoccerNet"
annotations_path = "/content/gdrive/MyDrive/datasets/SoccerNet_OSL_Annotations"
zip_folder = os.path.join(soccernet_root, "spotting-OSL/ResNET_PCA512/")

# Crea le cartelle necessarie
!mkdir -p "{soccernet_root}"
!mkdir -p "{annotations_path}"

# --- 1. DOWNLOAD ---
downloader = SoccerNetDownloader(LocalDirectory=soccernet_root)
print("Downloading features...")
downloader.downloadDataTask(task="spotting-OSL", split=["train", "valid", "test"], version="ResNET_PCA512")
print("\nDownloading labels...")
downloader.downloadGames(files=["Labels-v2.json"], split=["train", "valid", "test"])

# --- 2. DECOMPRESSIONE ---
zip_files = [os.path.join(zip_folder, f) for f in ["train.zip", "valid.zip", "test.zip"] if os.path.exists(os.path.join(zip_folder, f))]
for zip_file in zip_files:
    print(f"Decomprimo {zip_file}...")
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(soccernet_root)
print("\nDecompressione completata.")

# --- 3. CORREZIONE NOMI CARTELLE ---
print("\nInizio la correzione dei nomi delle cartelle...")
for league in os.listdir(soccernet_root):
    league_path = os.path.join(soccernet_root, league)
    if os.path.isdir(league_path) and league not in ["list_games", "spotting-OSL"]:
        for season in os.listdir(league_path):
            season_path = os.path.join(league_path, season)
            if os.path.isdir(season_path):
                for game_folder in [f for f in os.listdir(season_path) if "_" in f]:
                    original_path = os.path.join(season_path, game_folder)
                    new_name = game_folder.replace("_-_", " - ").replace("_", " ")
                    correct_path = os.path.join(season_path, new_name)
                    if os.path.isdir(correct_path):
                        for file_name in os.listdir(original_path):
                            if file_name.endswith('.npy'):
                                os.rename(os.path.join(original_path, file_name), os.path.join(correct_path, file_name))
                        if not os.listdir(original_path):
                            os.rmdir(original_path)
print("\nCorrezione dei nomi completata.")


# --- 4. CREAZIONE FILE ANNOTAZIONI ---
print("\nCreazione dei file JSON di annotazione...")
!python /content/OSL-ActionSpotting/tools/createJson/createjson_soccernet.py \
    --soccernet_root_dir "{soccernet_root}" \
    --save_dir "{annotations_path}"

print("\n\n✅ SETUP INIZIALE COMPLETATO. Ora puoi commentare questa cella e procedere al Passo 4.")

/content/gdrive/MyDrive/datasets/SoccerNet/spotting-OSL/ResNET_PCA512/train.zip already exists
/content/gdrive/MyDrive/datasets/SoccerNet/spotting-OSL/ResNET_PCA512/valid.zip already exists
/content/gdrive/MyDrive/datasets/SoccerNet/spotting-OSL/ResNET_PCA512/test.zip already exists

/content/gdrive/MyDrive/datasets/SoccerNet/england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley/Labels-v2.json already exists
/content/gdrive/MyDrive/datasets/SoccerNet/england_epl/2014-2015/2015-02-21 - 18-00 Crystal Palace 1 - 2 Arsenal/Labels-v2.json already exists
/content/gdrive/MyDrive/datasets/SoccerNet/england_epl/2014-2015/2015-02-21 - 18-00 Swansea 2 - 1 Manchester United/Labels-v2.json already exists
/content/gdrive/MyDrive/datasets/SoccerNet/england_epl/2014-2015/2015-02-22 - 19-15 Southampton 0 - 2 Liverpool/Labels-v2.json already exists
/content/gdrive/MyDrive/datasets/SoccerNet/england_epl/2015-2016/2015-08-08 - 19-30 Chelsea 2 - 2 Swansea/Labels-v2.json already exists
/content/gdr

In [5]:
# Cella di Correzione Definitiva

# Corregge il percorso di INPUT (il problema di prima)
!sed -i 's|/media/giancos/NewVolume/OSL-ActionSpotting|/content/gdrive/MyDrive/datasets|g' /content/OSL-ActionSpotting/tools/createJson/createjson_soccernet.py

# Corregge il percorso di OUTPUT (il problema attuale)
!sed -i 's|SoccerNetZip/Json_videos|SoccerNet_OSL_Annotations|g' /content/OSL-ActionSpotting/tools/createJson/createjson_soccernet.py

print("✅ Script createjson_soccernet.py patchato per i percorsi di input e output.")

✅ Script createjson_soccernet.py patchato per i percorsi di input e output.


In [4]:
# Cella per rieseguire la creazione dei JSON

soccernet_root = "/content/gdrive/MyDrive/datasets/SoccerNet"
annotations_path = "/content/gdrive/MyDrive/datasets/SoccerNet_OSL_Annotations"

print("Riprovo la creazione dei file JSON di annotazione...")
!python /content/OSL-ActionSpotting/tools/createJson/createjson_soccernet.py \
    --soccernet_root_dir "{soccernet_root}" \
    --save_dir "{annotations_path}"

print("\nProcesso di creazione dei JSON terminato.")

Riprovo la creazione dei file JSON di annotazione...
Traceback (most recent call last):
  File "/content/OSL-ActionSpotting/tools/createJson/createjson_soccernet.py", line 65, in <module>
    createjson("train", "train.json")
  File "/content/OSL-ActionSpotting/tools/createJson/createjson_soccernet.py", line 55, in createjson
    with open(JSON_all, 'w') as f:
         ^^^^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: '/content/gdrive/MyDrive/datasets/SoccerNetZip/Json_videos/train.json'

Processo di creazione dei JSON terminato.
